# Serverless Task Migration
This app is designed to help identify tasks running in your snowflake that are underutilizing a warehouse so that they can be changed to a serverless task. This is designed to migrate these tasks to improve the proficiency of these services as well as save on the cost of execution, as an under-utilized warehouse often charges for downtime when spun up for short tasks

This Notedbook will drop a streamlit file at SIT_SOLUTIONS.STM.CODE_STAGE and install the app SIT_SOLUTIONS.STM.STM_STREAMLIT.

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import pickle
import codecs
import os
import base64

from snowflake.snowpark.context import get_active_session
session = get_active_session()
session.sql(f"""ALTER SESSION SET QUERY_TAG = '{{"origin":"sf_sit","name":"sit_serveless_task_migration","version":{{"major":1, "minor":0}} }}'""").collect()

current_database = session.sql("SELECT CURRENT_DATABASE()").collect()[0][0]
current_schema = session.sql("SELECT CURRENT_schema()").collect()[0][0]
try:
    # session.sql("SELECT * from INSTALL_TEST.TEST.TEST_OPERATIONS").collect()
    st.session_state.opps_test_info = session.sql(f"SELECT * from {current_database}.{current_schema}.TEST_OPERATIONS").to_pandas()
    run_mode = "test"
except:
    st.success("Running Customer Install. Please follow the steps!")
    run_mode = "customer"

Please choose the location you'd like to install to

In [ ]:
if run_mode == 'test':
    opps_info = st.session_state.opps_test_info
    install_db = opps_info[opps_info["KEY"] == "INSTALL_DB"]["VALUE"].values[0]
    admin_schema = opps_info[opps_info["KEY"] == "ADMIN_SCHEMA"]["VALUE"].values[0]
    results_schema = opps_info[opps_info["KEY"] == "RESULT_SCHEMA"]["VALUE"].values[0]
    ops_schema = opps_info[opps_info["KEY"] == "OPS_SCHEMA"]["VALUE"].values[0]
    install_stage = opps_info[opps_info["KEY"] == "INSTALL_STAGE"]["VALUE"].values[0]
    ops_wh = opps_info[opps_info["KEY"] == "OPS_WH"]["VALUE"].values[0]
elif run_mode =="customer":
    install_db = st.text_input("Database")
    admin_schema = st.text_input("Admin Schema")
    results_schema = st.text_input("Results Schema")
    ops_schema = st.text_input("Operations Schema")
    install_stage = st.text_input("Install Stage (for hosting code files)")
    ops_wh = st.text_input("User Warehouse")

In [ ]:
CREATE OR REPLACE PROCEDURE {{install_db}}.{{admin_schema}}.PUT_TO_STAGE(STAGE VARCHAR, FILENAME VARCHAR, CONTENT VARCHAR)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION=3.8
PACKAGES=('snowflake-snowpark-python')
HANDLER='put_to_stage'
COMMENT='{"origin": "sf_sit","name": "sit_serveless_task_migration","version": "{major: 1, minor: 0}"}'
AS $$
import io
import os

def put_to_stage(session, stage, filename, content):
    local_path = '/tmp'
    local_file = os.path.join(local_path, filename)
    f = open(local_file, "w", encoding='utf-8')
    f.write(content)
    f.close()
    session.file.put(local_file, '@'+stage, auto_compress=False, overwrite=True)
    return "saved file "+filename+" in stage "+stage
$$;


In [ ]:
"""...
"""

In [ ]:
import pickle
import codecs
import os
import base64

from snowflake.snowpark.context import get_active_session
session = get_active_session()

def decode(obj):
    pickled = codecs.decode(obj.encode(), 'base64')
    return pickle.loads(pickled)


# Decode object to confirm it is the same as the original object
decoded_obj = decode(encoded_obj)

# Stage each file
for file_name, file_content in decoded_obj['files'].items():
    # replace double dollars to help with SQL compatibility
    file_content = str(file_content.decode("utf-8")).replace("$$", '"""')
    sql = """call {db}.{schema}.put_to_stage('{db}.{schema}.{stage}','{file_name}', $${file_content}$$)""".format(
        file_name=file_name, 
        file_content=file_content,
        db=install_db,
        schema=admin_schema,
        stage=install_stage)
    session.sql(sql).collect()

# Stage each image

In [ ]:
CREATE OR REPLACE STREAMLIT {{install_db}}.{{ops_schema}}.STM_STREAMLIT
ROOT_LOCATION = '@{{install_db}}.{{ops_schema}}.{{install_stage}}'
MAIN_FILE = 'serverless_task_migration.py'
QUERY_WAREHOUSE = {{ops_wh}}
COMMENT='{"origin": "sf_sit","name": "sit_serveless_task_migration","version": "{major: 1, minor: 0}"}';